<a href="https://colab.research.google.com/github/hwankang/Deep-Chemometrics/blob/master/27(fri)_Jan_5_00_pm_sgf_normal_R2_906_A_NIR_size_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.signal import savgol_filter
from sklearn.model_selection import train_test_split, cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename_a='/content/drive/MyDrive/machine_learning/A_NIR_DATA_csv.csv'
X2_df = pd.read_csv(filename_a,header=0, 
                   encoding="unicode-escape")
X2_df

In [ ]:
X=X2_df.values[:,3:].astype(float)
#X_train=X[110:,:]
#X_test=X[0:110,:]
y=X2_df.values[:,2:3].astype(float)
#y_train=Y[110:,:]
#y_test=Y[0:110,:]
#X_train
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.1, random_state=46)

In [ ]:
#from scipy.signal import savgol_filter
#X_train_f=X_train_n.astype(float)
#X_test_f=X_test_n.astype(float)
X_train_s = savgol_filter(X_train, 17, polyorder=2, deriv=1)
X_test_s = savgol_filter(X_test, 17, polyorder=2, deriv=1)

In [ ]:

def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
 
    return output_data

In [ ]:
X_train_r=snv(X_train_s)
X_test_r=snv(X_test_s)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise

In [ ]:
DENSE = 128
#DROPOUT = 0.025
DROPOUT = 0
C1_K  = 8 #Number of kernels/feature extractors for first layer
C1_S  = 32 #Width of the convolutional mini networks
C2_K  = 16
C2_S  = 32

#activation='elu'
activation='relu'
#activation='sigmoid'

input_dim = X_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    #model.add(GaussianNoise(0.05, input_shape=(input_dim,)))
    model.add(GaussianNoise(0.005, input_shape=(input_dim,)))
    ##model.add(GaussianNoise(0.004, input_shape=(input_dim,)))
    #First two is number of filter + kernel size
    model.add(Reshape((input_dim, 1)))
    #model.add(Conv1D(C1_K, (C1_S), activation=activation, border_mode='same'))
    #model.add(Conv1D(C2_K, (C2_S), border_mode='same', activation=activation))
    model.add(Conv1D(C1_K, (C1_S), activation=activation, padding='same'))
    model.add(Conv1D(C2_K, (C2_S), padding='same', activation=activation))
    model.add(Flatten())
    model.add(Dropout(DROPOUT))
    model.add(Dense(DENSE, activation=activation))
    model.add(Dense(1, activation='linear'))

    #model.compile(loss='mse', optimizer=keras.optimizers.Adadelta(lr=0.01))#, metrics=['mean_absolute_error'])
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.001))#, metrics=['mean_absolute_error'])

    return model

In [ ]:
model = make_model()
print(model.summary())

In [ ]:
rdlr = ReduceLROnPlateau(patience=25, factor=0.5, min_lr=1e-6, monitor='val_loss', verbose=1)

h = model.fit(X_train_r, y_train, epochs=600, batch_size=18*2, validation_data=(X_test_r, y_test), callbacks=[rdlr])

In [ ]:
plt.plot(h.history['loss'], label='loss')
plt.plot(h.history['val_loss'], label='val_loss')

plt.yscale('log')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
ax2 = plt.gca().twinx()
ax2.plot(h.history['lr'], color='r')
ax2.set_ylabel('lr',color='r')

_ = plt.legend()

In [ ]:
plt.scatter(y_train, model.predict(X_train_r))
plt.scatter(y_test, model.predict(X_test_r))
plt.plot([-5,3],[-5,3]) # Y = PredY line

In [ ]:
pred_Y_df=model.predict(X_test_r)
Y_df_true=y_test
from sklearn.metrics import mean_squared_error, r2_score
mse= mean_squared_error(Y_df_true, pred_Y_df)
rmse=np.sqrt(mse)
r2=r2_score(Y_df_true, pred_Y_df)
print('MSE : {: 0.3f}, || RMSE : {: 0.3f}, || R2 : {: 0.3f}'.format(mse,rmse,r2))

#end of the program

In [ ]:
X=X2_df.values[:,3:].astype(float)
keys=X2_df.keys()
key4=keys[3:].astype(float)
keys=list(key4)
wl=keys

In [ ]:
#insert GlobalStandardScaler
from __future__ import print_function
import numpy as np
import scipy #TODO reimplement as Numpy only
from scipy import newaxis as nA
##import scipy #TODO reimplement as Numpy only
##from scipy import newaxis as nA

class GlobalStandardScaler(object):
    """Scales to unit standard deviation and mean centering using global mean and std of X, skleran like API"""
    def __init__(self,with_mean=True, with_std=True, normfact=1.0):
        self._with_mean = with_mean
        self._with_std = with_std
        self.std = None
        self.normfact=normfact
        self.mean = None
        self._fitted = False
        
    def fit(self,X, y = None):
        X = np.array(X)
        self.mean = X.mean()
        self.std = X.std()
        self._fitted = True
        
    def transform(self,X, y=None):
        if self._fitted:
            X = np.array(X)
            if self._with_mean:
                X=X-self.mean
            if self._with_std:
                X=X/(self.std*self.normfact)
            return X
        else:
            print("Scaler is not fitted")
            return
            
    def inverse_transform(self,X, y=None):
        if self._fitted:
            X = np.array(X)
            if self._with_std:
                X=X*self.std*self.normfact
            if self._with_mean:
                X=X+self.mean
            return X
        else:
            print("Scaler is not fitted")
            return
            
    def fit_transform(self,X, y=None):
        self.fit(X)
        return self.transform(X)




In [ ]:
xscaler = GlobalStandardScaler()

#Calibrate is smaler than test, so they are swapped
X_train = xscaler.fit_transform(X) #From instrument 1
#X_test = xscaler.transform(X_test_a) #! NB only transform on test set. From instrument 2

In [ ]:
plt.plot(wl,X.T)
plt.xlabel('nm')
plt.ylabel('normalized intensity')
plt.title('Normalized training data')
plt.show()

In [ ]:
xscaler = GlobalStandardScaler()
X_2=X2_df.values[:,3:].astype(float)
#Calibrate is smaler than test, so they are swapped
X_c = xscaler.fit_transform(X_2) #From instrument 1
#X_test = xscaler.transform(X_test_a) #! NB only transform on test set. From instrument 2

In [ ]:
plt.plot(wl,X_c.T)
plt.xlabel('nm')
plt.ylabel('normalized intensity')
plt.title('Normalized training data')
plt.show()

In [ ]:
#use
import scipy #TODO reimplement as Numpy only
from scipy import newaxis as nA
class EmscScaler(object):
    def __init__(self,order=1):
        self.order = order
        self._mx = None
        
    def mlr(self,x,y):
        """Multiple linear regression fit of the columns of matrix x 
        (dependent variables) to constituent vector y (independent variables)
        
        order -     order of a smoothing polynomial, which can be included 
                    in the set of independent variables. If order is
                    not specified, no background will be included.
        b -         fit coeffs
        f -         fit result (m x 1 column vector)
        r -         residual   (m x 1 column vector)
        """
        
        if self.order > 0:
            s= np.ones((len(y),1))
            for j in range(self.order):
                s= np.concatenate((s,(np.arange(0,1+(1.0/(len(y)-1)),1.0/(len(y)-1))**j)[:,nA]),1)
            X= np.concatenate((x, s),1)
        else:
            X = x
        b = np.dot(np.dot(np.linalg.pinv(np.dot(np.transpose(X),X)),np.transpose(X)),y)
        f = np.dot(X,b)
        r = y - f

        return b,f,r

    
    def inverse_transform(self, X, y=None):
        print("Warning: inverse transform not possible with Emsc")
        return X
    
    def fit(self, X, y=None):
        """fit to X (get average spectrum), y is a passthrough for pipeline compatibility"""
        self._mx = np.mean(X,axis=0)[:,nA]
        
    def transform(self, X, y=None, copy=None):
        if type(self._mx) == type(None):
            print("EMSC not fit yet. run .fit method on reference spectra")
        else:
            #do fitting
            corr = np.zeros(X.shape)
            for i in range(len(X)):
                b,f,r = self.mlr(self._mx, X[i,:][:,nA])
                corr[i,:] = np.reshape((r/b[0,0]) + self._mx, (corr.shape[1],))
            return corr
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)
          

In [ ]:
# Use
#EMSC rescaling
##from ChemUtils import EmscScaler

emsc = EmscScaler()

emsc.fit(X_train)

#X_train_emsc = emsc.transform(X_train_a)
X_train_emsc = emsc.fit_transform(X_train)

#_ = plt.plot(dataset['axisscale'],X_train_emsc.T)
#X_train_emsc[0,:]

In [ ]:
plt.plot(wl,X_train_emsc.T)
plt.xlabel('nm')
plt.ylabel('normalized intensity')
plt.title('Normalized training data')
plt.show()

In [ ]:
#delete
import scipy #TODO reimplement as Numpy only
from scipy import newaxis as nA
class SavGolFilt(object):
    """Applies a Savitsky-Golay filter of order k and frame width F.
    The order must be odd and the frame width (F) a positive integer of
    a value greater than k
    """
    #TODO use the scipy implementation
    def __init__(self, order=1, width=11):
        self.k = order
        self.frame = width

    def transform(self,myarray,y=None):
        """Applies a Savitsky-Golay filter of order k and frame width F.
        The order must be odd and the frame width (F) a positive integer of
        a value greater than k
        """
        frange = scipy.arange(-(self.frame-1)/2,((self.frame-1)/2)+1)
        f, vande = 0, scipy.zeros((self.frame,self.frame))
        while f < self.frame:    # compute Vandemonde matrix
            vande[f,:] = frange**f
            f = f+1
        vande = scipy.transpose(vande,(1,0))
        vande = vande[:,0:self.k+1]
        Q,R = scipy.linalg.qr(vande,vande.shape[1]) # Do QR decomposition
    
    #    print vande.shape
    #    print Q.shape
    #    print R[0:vande.shape[1]]
        G = scipy.dot(vande,scipy.dot(scipy.linalg.inv(R[0:vande.shape[1]]), 
        scipy.transpose(scipy.linalg.inv(R[0:vande.shape[1]])))) # Find the matrix of differentiators
    
        B = scipy.dot(G,scipy.transpose(vande)) # Projection matrix
    
        myarray = scipy.transpose(myarray)
        extract_array, extract_B = myarray[0:self.frame,:], B[(((self.frame-1)/2)+1):self.frame,:]
        start_array = scipy.dot(extract_B[::-1],extract_array[::-1]) # first bins

        array_size = myarray.shape
        last, mid_array = (self.frame-1)/2, scipy.zeros((array_size[0],array_size[1]),'d')
        extract_B = scipy.reshape(B[((self.frame-1)/2),:],(self.frame,1))
        while last < array_size[0]-((self.frame-1)/2):
            mid_array[last,:] = sum((extract_B*myarray[last-((self.frame-1)/2):last+((self.frame-1)/2)+1,:]),0) #middle bit
            last = last+1
        
        extract_array, extract_B = myarray[array_size[0]-self.frame:array_size[0],:], B[0:(self.frame-1)/2,:]
        end_array = scipy.dot(extract_B[::-1],extract_array[::-1]) # last bins

        mid_array[0:(self.frame-1)/2,:], mid_array[array_size[0]-((self.frame-1)/2):array_size[0],:] = start_array, end_array
        return scipy.transpose(mid_array)

    def fit(self, X,y=None):
        print("Fit not needed for filter")
        pass
        
    def fit_transform(self, X,y=None):
        return self.transform(X)


        

In [ ]:
# Use
#EMSC rescaling
##from ChemUtils import EmscScaler
from scipy import linalg

sgf = SavGolFilt()

sgf.fit(X)

#X_train_emsc = emsc.transform(X_train_a)
X_sgf = sgf.transform(X)

#_ = plt.plot(dataset['axisscale'],X_train_emsc.T)
#X_train_emsc[0,:]

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
X2 = savgol_filter(X, 11, polyorder=2, deriv=1)
plt.plot(wl,X2.T)
plt.xlabel('nm')
plt.ylabel('normalized intensity')
plt.title('SGFilter data')
plt.show()